In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
import optuna
import pandas as pd
import numpy as np
from graphviz import Digraph

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 20)

# Cteate study object

In [6]:
study = optuna.create_study(storage='sqlite:///optuna/pre-train/ResNet32/optuna.db',
                            study_name='experiment01',
                            #pruner=pruner,
                            direction="minimize",
                            load_if_exists=True)

[I 2024-07-08 15:51:20,682] Using an existing study with name 'experiment01' instead of creating a new one.


# Watch

In [7]:
optuna.visualization.plot_intermediate_values(study)

In [10]:
import pandas as pd
from tabulate import tabulate
import math

def format_value(value, is_loguniform=False):
    if is_loguniform:
        exp = math.floor(math.log10(abs(value)))
        mantissa = value / (10 ** exp)
        return f"{mantissa:.2f}e{exp}"
    return f"{value:.4f}"

def format_study_results(df):
    df_sorted = df.sort_values('value', ascending=True)
    results = []
    
    for i in range(len(df_sorted)):
        row = df_sorted.iloc[i]
        results.append({
            'Accuracy': f"{100 - row.value:.2f}%" if row.value is not None else f"{None}",
            'Learning Rate': format_value(row.params_lr, is_loguniform=True),
            'Weight Decay': format_value(row.params_weight_decay, is_loguniform=True),
            'Beta1': f"{row.params_beta1:.4f}",
            'AMSGrad': str(row.params_amsgrad),
            'Duration': str(row.duration).split()[-1]  # Only show time part
        })
    
    table = tabulate(results, headers='keys', tablefmt='pretty', numalign='right')
    
    return table

print(format_study_results(study.trials_dataframe()))

+----------+---------------+--------------+--------+---------+-----------------+
| Accuracy | Learning Rate | Weight Decay | Beta1  | AMSGrad |    Duration     |
+----------+---------------+--------------+--------+---------+-----------------+
|  72.39%  |    1.29e-2    |   6.21e-2    | 0.9053 |  True   | 00:39:15.649048 |
|  72.19%  |    1.17e-2    |   5.59e-2    | 0.8912 |  True   | 00:39:19.706052 |
|  72.10%  |    9.05e-3    |   5.54e-2    | 0.9125 |  True   | 00:37:05.347532 |
|  72.08%  |    9.59e-3    |   8.30e-2    | 0.9132 |  True   | 00:39:18.059372 |
|  71.91%  |    1.26e-2    |   5.42e-2    | 0.9045 |  True   | 00:39:21.663228 |
|  71.90%  |    1.13e-2    |   5.82e-2    | 0.9041 |  True   | 00:39:16.464826 |
|  71.82%  |    1.32e-2    |   4.50e-2    | 0.8992 |  True   | 00:39:25.264150 |
|  71.70%  |    1.24e-2    |   5.60e-2    | 0.9058 |  True   | 00:39:19.793147 |
|  71.66%  |    1.06e-2    |   5.45e-2    | 0.8865 |  True   | 00:39:18.656371 |
|  71.63%  |    1.34e-2    |

# Graph

In [ ]:
num_models = 3
epochs = 200
target_model = "ResNet32"

In [ ]:
df = study.trials_dataframe()
df = df[["number", "state", "value", "datetime_start","datetime_complete","params","user_attrs"]]

complete = df[df["state"] == optuna.structs.TrialState.COMPLETE]
params = complete["params"]

model = params.loc[:,params.columns.str.contains("model")]

model_acc = []
for id_ in complete["number"]:
    model_acc += [[pd.read_csv(f"./result/{id_:04d}/log/net{i}/epoch_log.csv",
                               index_col="Unnamed: 0").at[epochs, "test_accuracy"] for i in range(num_models)]]
    
model_acc = pd.DataFrame(model_acc, index=model.index, columns=[f"model_{i}_acc" for i in range(num_models)])

sorted_df = complete.sort_values(by="value")
sorted_acc = sorted_df["value"]

sorted_df = pd.concat([sorted_acc, model.loc[sorted_acc.index], model_acc.loc[sorted_acc.index]], axis=1).rename(columns={0:"max_accuracy"})
sorted_df

In [ ]:
top = 0

In [ ]:
### Loss
loss = params.loc[:, params.columns.str.contains("loss")].loc[sorted_df.index[top]]
model_name = params.loc[:, params.columns.str.contains("model")].loc[sorted_df.index[top]]

wh = int(np.sqrt(len(loss)))
df_loss = pd.DataFrame(loss.values.reshape((wh,wh)), columns=model_name, index=model_name)
df_loss

### Gate
gate = params.loc[:, params.columns.str.contains("gate")].loc[sorted_df.index[top]]
df_gate = pd.DataFrame(df_loss.copy()).applymap(lambda x :None)

for gate_name, val in gate.to_dict().items():
    source, target, _ = gate_name.split("_")
    df_gate.iloc[int(source), int(target)] = val

edge_color = {
    "ThroughGate": "3", 
    "LinearGate": "1", 
    "CorrectGate": "2"
}

G = Digraph(format="pdf", engine="dot")

acc = model_acc.loc[sorted_acc.index].iloc[top]
for target in range(len(df_loss)):    
    G.node(f"{target+1}. "+df_loss.index[target]+f" ({acc[target]}%)",
           color='gray90', fillcolor='gray90', style='filled')
G.node(f"{1}. "+df_loss.index[0]+f" ({acc[0]}%)", color='pink', fillcolor='pink', style='radial')
    
for target in range(len(df_loss)):
    for source in range(len(df_loss)):
        gate = df_gate.iloc[target,source]
        if gate != "CutoffGate":
            label = df_gate.iloc[target, source].replace("Gate","")                
            if source == target:
                if gate == "CorrectGate":
                    gate = "ThroughGate"
                    label = "Through"
                G.edge(f"{target}",
                       f"{target+1}. "+df_loss.index[target]+f" ({acc[target]}%)",
                       label=label, fontsize="13", fontcolor=edge_color[gate],
                       color=edge_color[gate], colorscheme="dark28")
                G.node(f"{target}", label="Label", color='white', style='filled')
            else:
                G.edge(f"{source+1}. "+df_loss.columns[source]+f" ({acc[source]}%)",
                       f"{target+1}. "+df_loss.index[target]+f" ({acc[target]}%)",
                       label=label, fontsize="13", fontcolor=edge_color[gate],
                       color=edge_color[gate], colorscheme="dark28")

                
for target in range(len(df_loss)):
    if (df_gate.iloc[target] == "CutoffGate").all():
        G.node(f"{target+1}. "+df_loss.columns[target]+f" ({acc[target]}%)",
               color='lightblue',fillcolor='lightblue', style='radial')

G.render(filename=f"{top}", directory="./topn_graph", cleanup=True, format="pdf")
G